In [1]:
!pip install pyxlsb

In [2]:
import pandas as pd
import pyxlsb
import numpy as np

In [4]:
dataframe = pd.read_excel("/content/stc TV Data Set_T3.xlsb",index_col=0)

In [5]:
dataframe.shape

(1048575, 5)

In [6]:
dataframe.head()

,user_id_maped,program_name,rating,date_,program_genre
0,26138,100 treets,1,2017-05-27,Drama
1,7946,Moana,1,2017-05-21,Animation
2,7418,The Mermaid Princess,1,2017-08-10,Animation
3,19307,The Mermaid Princess,2,2017-07-26,Animation
4,15860,Churchill,2,2017-07-07,Biography


In [7]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   user_id_maped  1048575 non-null  int64         
 1   program_name   1048575 non-null  object        
 2   rating         1048575 non-null  int64         
 3   date_          1048575 non-null  datetime64[ns]
 4   program_genre  1048575 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 48.0+ MB


In [8]:
dataframe.isnull().any() # Check the values are null

user_id_maped    False
program_name     False
rating           False
date_            False
program_genre    False
dtype: bool

In [14]:
# Calculate the average rating for each program in the DataFrame dataframe, sorts them in descending order based on their average ratings, and stores the result in the variable program_ratings.
program_ratings = dataframe.groupby('program_name')['rating'].mean().sort_values(ascending=False)
top_programs = program_ratings.head(5)  # Get top 5 programs by average rating
print(top_programs)

program_name
World's Best Chefs  Alex Atala                  4.0
Criminal Minds     The Lesson                   4.0
Criminal Minds     Children Of The Dark         4.0
Scandal     candal     Enemy Of The tate        4.0
Scandal     candal     Grant: For The People    4.0
Name: rating, dtype: float64


In [15]:
movie_df=dataframe.pivot_table(index='program_name',columns='user_id_maped',values='rating').fillna(0)
movie_df.head()

user_id_maped,1,5,9,11,15,17,20,26,28,30,...,34259,34261,34263,34265,34267,34269,34271,34273,34277,34280
program_name,,,,,,,,,,,,,,,,,,,,,
#FollowFriday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Days in a Madhouse,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100 treets,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101 Dalmatians,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102 Dalmatians,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
movie_df_matrix = csr_matrix(movie_df.values)
model = NearestNeighbors(metric = 'cosine', algorithm = 'brute') # Define the model
model.fit(movie_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [17]:
movie_df_select= movie_df.reset_index()
movie_df_select[['program_name']]
dataframe[['program_name']].head(10) # Get the top 10 movies

,program_name
0,100 treets
1,Moana
2,The Mermaid Princess
3,The Mermaid Princess
4,Churchill
5,Beavis And Butt-Head Do America
6,The Mermaid Princess
7,Coco
8,Kidnap
9,The Accountant


In [19]:
recommendations = 5

# Find the index of the program in the dataframe
program_index = movie_df_select.index[movie_df_select['program_name'] == 'Moana'].tolist()[0]

# Query the model for nearest neighbors
distances, indices = model.kneighbors(movie_df.iloc[program_index].values.reshape(1, -1), n_neighbors=recommendations+1)

# Print recommendations
print(f"Recommendations for {'Moana'}:\n")
for i in range(1, len(distances.flatten())):
    print(f"{i}: {movie_df.index[indices.flatten()[i]]}, with distance of {distances.flatten()[i]:.2f}")

Recommendations for Moana:

1: Trolls, with distance of 0.43
2: Surf's Up : WaveMania, with distance of 0.47
3: The Mermaid Princess, with distance of 0.51
4: The Boss Baby, with distance of 0.55
5: The Jetsons & WWE: Robo-WrestleMania!, with distance of 0.56
